In [ ]:
# =============================================================================
# 02_data_preparation.ipynb
# =============================================================================
# FarmTech Solutions - Análise de Rendimento de Safra
# EDA Completa: Análise Exploratória dos Dados
# Versão: 1.0
# =============================================================================

## 📊 Objetivo da Preparação de Dados

### Contexto Específico do Projeto FarmTech Solutions

**Situação Atual:**
Este notebook realiza a **Preparação de Dados** como segunda etapa do projeto FarmTech Solutions, desenvolvido para a **Fase 5 do curso de Inteligência Artificial da FIAP**. Com base nas descobertas da EDA, preparamos os dados para as etapas subsequentes de clustering e modelagem preditiva.

**Descobertas da EDA que Orientam a Preparação:**
- **Yield bimodal por cultura**: Oil palm (~175k), Rice (~32k), Cocoa/Rubber (~8k)
- **Variáveis ambientais idênticas**: Mesmas condições climáticas para todas as culturas
- **Correlações ambientais**: Precipitação ↔ Umidade (0.749), Temperatura ↔ Umidade específica (0.699), Temperatura ↔ Umidade Relativa (-0.337)

- **Ausência de correlação linear**: Variáveis ambientais não se correlacionam linearmente com Yield
- **Dados de qualidade**: 0 valores faltantes, 0 duplicatas, tipos adequados

**Dataset Específico:**
- **156 registros** de 4 culturas (Cocoa, Oil palm fruit, Rice paddy, Rubber)
- **6 variáveis**: Crop, Precipitation, Specific Humidity, Relative Humidity, Temperature, Yield
- **Variável alvo**: Yield (rendimento em toneladas/hectare)

### Metodologia e Objetivos

**Abordagem Metodológica:**
Seguimos uma progressão sistemática baseada nas descobertas da EDA:

1. **Análise de Outliers**: Validação crítica dos valores extremos identificados na EDA
2. **Feature Engineering**: Criação de variáveis derivadas para capturar relações não-lineares
3. **Encoding**: Transformação da variável categórica 'Crop' para algoritmos de ML
4. **Normalização**: Padronização para algoritmos sensíveis à escala

**Justificativa das Decisões Técnicas:**
- **Análise de Outliers**: EDA identificou 12 outliers em Temperature e 35 em Yield - necessário validar se são reais ou falsos positivos
- **Feature Engineering**: Criar interações entre variáveis ambientais correlacionadas para capturar padrões não-lineares
- **One-Hot Encoding**: Variável 'Crop' é categórica e crucial para prever Yield
- **StandardScaler**: Necessário para algoritmos como SVM, Neural Networks, K-means

**Processo de Descoberta:**
Abordagem baseada em evidências da EDA. Cada transformação é justificada pelas descobertas da análise exploratória anterior.

## 1. 🚀 Setup e Imports

### Bibliotecas e Configurações

**Bibliotecas Essenciais:**
- **pandas & numpy**: Manipulação e operações numéricas para dataset agrícola (156 registros)
- **scikit-learn**: Transformações de dados (StandardScaler, LabelEncoder, SelectKBest)
- **matplotlib & seaborn**: Visualizações para validação das transformações

**Justificativa das Decisões:**
- **StandardScaler**: Normalizar features preditoras (Precipitation, Temperature, Humidity) para algoritmos sensíveis à escala
- **Yield mantido original**: Variável alvo não deve ser normalizada
- **LabelEncoder**: Alternativa ao One-Hot Encoding para variável 'Crop'
- **SelectKBest**: Seleção de features baseada em testes estatísticos

**Conexão com EDA:**
As bibliotecas selecionadas atendem às necessidades identificadas na análise exploratória: normalização para alta variabilidade, encoding para variável categórica crucial, e visualizações para validar transformações.

In [15]:
# =============================================================================
# SETUP E IMPORTS
# =============================================================================

# Importar bibliotecas essenciais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression

# Configurações de visualização
plt.style.use('default')
sns.set_palette("husl")

print("🔧 Setup da preparação de dados concluído!")
print("✅ Bibliotecas importadas com sucesso!")

🔧 Setup da preparação de dados concluído!
✅ Bibliotecas importadas com sucesso!


## 2.1 📥 Carregamento e Verificação Inicial

### Objetivo da Seção
Realizar verificação sistemática dos dados para identificar:
- **Integridade**: Valores faltantes, duplicatas e inconsistências
- **Qualidade**: Tipos de dados e ranges de valores
- **Consistência**: Valores dentro de limites esperados
- **Base**: Preparação para próximas etapas de limpeza

### Estratégia de Verificação:
- **Análise de valores faltantes**: Identificar e quantificar dados ausentes
- **Verificação de duplicatas**: Detectar observações repetidas
- **Análise de tipos**: Confirmar tipos corretos de dados
- **Verificação de ranges**: Validar limites mínimos e máximos

In [58]:
# =============================================================================
# CARREGAMENTO E VERIFICAÇÃO INICIAL DOS DADOS
# =============================================================================

print("📊 Carregando dados para preparação...")

# Carregar dados
df = pd.read_csv('../data/raw/crop_yield.csv')

print(f"✅ Dataset carregado: {df.shape[0]} registros, {df.shape[1]} variáveis")

# Verificação de integridade
print("\n🔍 VERIFICAÇÃO DE INTEGRIDADE:")
print(f"   → Valores faltantes: {df.isnull().sum().sum()}")
print(f"   → Duplicatas: {df.duplicated().sum()}")
print(f"   → Tipos de dados: {df.dtypes.to_dict()}")

# Verificação de consistência
print("\n📊 VERIFICAÇÃO DE CONSISTÊNCIA:")
for col in df.select_dtypes(include=[np.number]).columns:
    print(f"   → {col}: {df[col].min():.2f} a {df[col].max():.2f}")

print("✅ Verificação inicial concluída!")

📊 Carregando dados para preparação...
✅ Dataset carregado: 156 registros, 6 variáveis

🔍 VERIFICAÇÃO DE INTEGRIDADE:
   → Valores faltantes: 0
   → Duplicatas: 0
   → Tipos de dados: {'Crop': dtype('O'), 'Precipitation (mm day-1)': dtype('float64'), 'Specific Humidity at 2 Meters (g/kg)': dtype('float64'), 'Relative Humidity at 2 Meters (%)': dtype('float64'), 'Temperature at 2 Meters (C)': dtype('float64'), 'Yield': dtype('int64')}

📊 VERIFICAÇÃO DE CONSISTÊNCIA:
   → Precipitation (mm day-1): 1934.62 a 3085.79
   → Specific Humidity at 2 Meters (g/kg): 17.54 a 18.70
   → Relative Humidity at 2 Meters (%): 82.11 a 86.10
   → Temperature at 2 Meters (C): 25.56 a 26.81
   → Yield: 5249.00 a 203399.00
✅ Verificação inicial concluída!


## 2.2 🔍 Análise de Outliers

### Contexto e Justificativa

**Descobertas da EDA que Orientam a Análise:**
A EDA identificou outliers potenciais em duas variáveis críticas:
- **Temperature**: 12 outliers detectados pelo método IQR
- **Yield**: 35 outliers detectados pelo método IQR

**Problema Específico:**
Necessário validar se esses outliers são:
- **Valores reais** representando condições climáticas excepcionais
- **Falsos positivos** devido à natureza bimodal do Yield por cultura
- **Erros de coleta** que podem comprometer a modelagem

**Impacto na Modelagem:**
Outliers podem distorcer algoritmos sensíveis (SVM, Neural Networks) e afetar a precisão preditiva do sistema FarmTech Solutions.

### Metodologia e Estratégia

**Abordagem Metodológica:**
Seguimos uma estratégia sistemática para validação crítica:

1. **Método IQR**: Detecção baseada em quartis (Q1 - 1.5×IQR, Q3 + 1.5×IQR)
2. **Método Z-Score**: Detecção baseada em desvios padrão (|z| > 3)
3. **Análise contextual**: Considerar significado agrícola dos valores extremos
4. **Decisão baseada em evidências**: Não remover automaticamente, analisar contexto

**Justificativa das Decisões Técnicas:**
- **Dois métodos diferentes**: Validar consistência entre abordagens estatísticas
- **Análise contextual**: Yield bimodal pode gerar falsos positivos (Oil palm vs outras culturas)
- **Decisão inteligente**: Manter dados valiosos ao invés de remoção automática
- **Foco em agricultura**: Considerar se outliers representam condições climáticas excepcionais

**Objetivos Específicos:**
- Validar se outliers em Temperature representam condições climáticas excepcionais
- Determinar se outliers em Yield são resultado da distribuição bimodal por cultura
- Decidir estratégia de tratamento baseada em evidências estatísticas e contexto agrícola

In [59]:
# =============================================================================
# ANÁLISE DE OUTLIERS - DETECÇÃO E ANÁLISE
# =============================================================================

print("🔍 Iniciando análise de outliers...")

# Selecionar variáveis numéricas
numeric_vars = ['Precipitation (mm day-1)', 'Specific Humidity at 2 Meters (g/kg)',
                'Relative Humidity at 2 Meters (%)', 'Temperature at 2 Meters (C)', 'Yield']

# Método IQR para detecção de outliers
outliers_iqr = {}
outliers_zscore = {}

for var in numeric_vars:
    # Método IQR
    Q1 = df[var].quantile(0.25)
    Q3 = df[var].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_iqr[var] = df[(df[var] < lower_bound) | (df[var] > upper_bound)]
    
    # Método Z-Score
    z_scores = np.abs((df[var] - df[var].mean()) / df[var].std())
    outliers_zscore[var] = df[z_scores > 3]
    
    print(f"\n📊 {var}:")
    print(f"   → Outliers (IQR): {len(outliers_iqr[var])} ({len(outliers_iqr[var])/len(df)*100:.1f}%)")
    print(f"   → Outliers (Z-Score): {len(outliers_zscore[var])} ({len(outliers_zscore[var])/len(df)*100:.1f}%)")
    print(f"   → Limites IQR: {lower_bound:.2f} a {upper_bound:.2f}")

print("✅ Análise de outliers concluída!")

🔍 Iniciando análise de outliers...

📊 Precipitation (mm day-1):
   → Outliers (IQR): 0 (0.0%)
   → Outliers (Z-Score): 0 (0.0%)
   → Limites IQR: 1680.35 a 3340.72

📊 Specific Humidity at 2 Meters (g/kg):
   → Outliers (IQR): 0 (0.0%)
   → Outliers (Z-Score): 0 (0.0%)
   → Limites IQR: 17.48 a 18.95

📊 Relative Humidity at 2 Meters (%):
   → Outliers (IQR): 0 (0.0%)
   → Outliers (Z-Score): 0 (0.0%)
   → Limites IQR: 82.03 a 87.59

📊 Temperature at 2 Meters (C):
   → Outliers (IQR): 12 (7.7%)
   → Outliers (Z-Score): 0 (0.0%)
   → Limites IQR: 25.60 a 26.72

📊 Yield:
   → Outliers (IQR): 35 (22.4%)
   → Outliers (Z-Score): 0 (0.0%)
   → Limites IQR: -80458.75 a 156305.25
✅ Análise de outliers concluída!


In [60]:
# =============================================================================
# RESULTADOS DA ANÁLISE DE OUTLIERS
# =============================================================================

print("\n" + "="*60)
print("📊 RESULTADOS DA ANÁLISE DE OUTLIERS")
print("="*60)

print("🔍 COMPARAÇÃO DOS MÉTODOS:")
print("   → Precipitation: 0 outliers (ambos os métodos)")
print("   → Specific Humidity: 0 outliers (ambos os métodos)")
print("   → Relative Humidity: 0 outliers (ambos os métodos)")
print("   → Temperature: 12 outliers (IQR) vs 0 outliers (Z-Score)")
print("   → Yield: 35 outliers (IQR) vs 0 outliers (Z-Score)")

print("\n📊 ANÁLISE CRÍTICA:")
print("   ⚠️ Discordância entre métodos para Temperature e Yield")
print("   → IQR: Muito conservador, range muito estreito")
print("   → Z-Score: Mais realista, baseado em distribuição normal")
print("   → Limite negativo em Yield: Matematicamente impossível")

print("\n🌾 CONTEXTO AGRÍCOLA:")
print("   ✅ Yield bimodal: Característica natural (Oil palm vs outros)")
print("   ✅ Temperature: Variação normal para agricultura tropical")
print("   ✅ Variações ambientais: Dentro de ranges esperados")

print("\n💡 DECISÃO ESTRATÉGICA:")
print("   → NÃO remover outliers baseado em Z-Score (0 outliers)")
print("   → Manter todos os dados como informação valiosa")
print("   → Focar em feature engineering para capturar padrões")

print("✅ Resultados da análise de outliers concluídos!")


📊 RESULTADOS DA ANÁLISE DE OUTLIERS
🔍 COMPARAÇÃO DOS MÉTODOS:
   → Precipitation: 0 outliers (ambos os métodos)
   → Specific Humidity: 0 outliers (ambos os métodos)
   → Relative Humidity: 0 outliers (ambos os métodos)
   → Temperature: 12 outliers (IQR) vs 0 outliers (Z-Score)
   → Yield: 35 outliers (IQR) vs 0 outliers (Z-Score)

📊 ANÁLISE CRÍTICA:
   ⚠️ Discordância entre métodos para Temperature e Yield
   → IQR: Muito conservador, range muito estreito
   → Z-Score: Mais realista, baseado em distribuição normal
   → Limite negativo em Yield: Matematicamente impossível

🌾 CONTEXTO AGRÍCOLA:
   ✅ Yield bimodal: Característica natural (Oil palm vs outros)
   ✅ Temperature: Variação normal para agricultura tropical
   ✅ Variações ambientais: Dentro de ranges esperados

💡 DECISÃO ESTRATÉGICA:
   → NÃO remover outliers baseado em Z-Score (0 outliers)
   → Manter todos os dados como informação valiosa
   → Focar em feature engineering para capturar padrões
✅ Resultados da análise de out

## 2.3 🔧 Feature Engineering

### Contexto e Justificativa

**Descobertas da EDA que Orientam o Feature Engineering:**
A análise exploratória revelou padrões que justificam a criação de features derivadas:

1. **Correlações ambientais identificadas**:
   - Precipitação ↔ Umidade Relativa: 0.749 (forte correlação)
   - Temperatura ↔ Umidade Específica: 0.699 (forte correlação)
   - Temperatura ↔ Umidade Relativa: -0.337 (correlação fraca negativa)

2. **Ausência de correlação linear com Yield**:
   - Variáveis ambientais individuais não se correlacionam com rendimento
   - Necessidade de capturar relações não-lineares e interações

3. **Distribuição bimodal do Yield**:
   - Oil palm (175k) vs outras culturas (8-32k)
   - Possível existência de relações complexas entre variáveis ambientais

**Problema Específico:**
Algoritmos de ML podem não capturar automaticamente:
- Interações entre variáveis correlacionadas
- Relações não-lineares entre variáveis ambientais e Yield
- Padrões complexos que não são óbvios nas variáveis originais

### Metodologia e Estratégia

**Abordagem Metodológica:**
Seguimos uma estratégia sistemática baseada em evidências da EDA:

**1. Criação de Features:**
- **Interações Ambientais**: Temperature × Humidity, Precipitation × Temperature
- **Índices Climáticos**: Thermal Comfort, Effective Humidity, Growth Conditions
- **Transformações**: Log Precipitation, Temperature², Humidity²

**2. Validação de Qualidade:**
- **Análise de Correlações**: Verificar redundâncias com features originais
- **Critério de Seleção**: Manter apenas features com correlação < 0.95
- **Validação Agrícola**: Confirmar significado prático das features

**3. Processo Iterativo:**
- **Criar** → **Validar** → **Manter/Remover** → **Repetir**

**Justificativa das Decisões Técnicas:**
- **Validação sistemática**: Evitar multicolinearidade e redundâncias
- **Critério objetivo**: Correlação < 0.95 para manter features
- **Processo iterativo**: Garantir qualidade de cada feature criada
- **Validação agrícola**: Confirmar relevância prática para agricultura

**Objetivos Específicos:**
- Criar features que capturem interações entre variáveis ambientais
- Validar qualidade de cada feature através de análise de correlações
- Manter apenas features úteis e não redundantes
- Preparar dataset otimizado para algoritmos de ML

In [61]:
# =============================================================================
# FEATURE ENGINEERING - CRIAÇÃO E VALIDAÇÃO DE VARIÁVEIS DERIVADAS
# =============================================================================

print("🔧 Iniciando Feature Engineering com validação...")

# Backup dos dados originais
df_original = df.copy()
features_removed = []

# Separar features e target para contagem correta
original_features = [col for col in df.columns if col != 'Yield']
target = 'Yield'

print(f"�� Estado inicial:")
print(f"   → Features preditoras originais: {len(original_features)}")
print(f"   → Target: {target}")
print(f"   → Total de colunas: {len(df.columns)}")

# Função para validar correlação
def validate_feature(df, new_feature, original_feature, threshold=0.95):
    """Valida se nova feature não é redundante com feature original"""
    correlation = df[new_feature].corr(df[original_feature])
    print(f"   → Correlação {new_feature} ↔ {original_feature}: {correlation:.3f}")
    
    if abs(correlation) > threshold:
        print(f"   ❌ Feature redundante (correlação > {threshold}), removendo...")
        df = df.drop(new_feature, axis=1)
        features_removed.append(new_feature)
        return df, False
    else:
        print(f"   ✅ Feature mantida (correlação ≤ {threshold})")
        return df, True

# 1. INTERAÇÕES AMBIENTAIS
print("\n📊 1. Criando interações ambientais...")

# Temperature × Relative Humidity
df['temp_humidity_interaction'] = df['Temperature at 2 Meters (C)'] * df['Relative Humidity at 2 Meters (%)']
df, kept = validate_feature(df, 'temp_humidity_interaction', 'Temperature at 2 Meters (C)')

# Precipitation × Temperature
df['precip_temp_ratio'] = df['Precipitation (mm day-1)'] / (df['Temperature at 2 Meters (C)'] + 1)
df, kept = validate_feature(df, 'precip_temp_ratio', 'Precipitation (mm day-1)')

# Specific × Relative Humidity
df['humidity_combined'] = df['Specific Humidity at 2 Meters (g/kg)'] * df['Relative Humidity at 2 Meters (%)'] / 100
df, kept = validate_feature(df, 'humidity_combined', 'Specific Humidity at 2 Meters (g/kg)')

print("   ✅ Interações ambientais validadas")

# 2. ÍNDICES CLIMÁTICOS
print("\n🌡️ 2. Criando índices climáticos...")

# Thermal Comfort
df['thermal_comfort'] = 0.5 * (df['Temperature at 2 Meters (C)'] + df['Relative Humidity at 2 Meters (%)'])
df, kept = validate_feature(df, 'thermal_comfort', 'Temperature at 2 Meters (C)')

# Effective Humidity
df['effective_humidity'] = df['Specific Humidity at 2 Meters (g/kg)'] * df['Relative Humidity at 2 Meters (%)'] / 100
df, kept = validate_feature(df, 'effective_humidity', 'Specific Humidity at 2 Meters (g/kg)')

# Growth Conditions
df['growth_conditions'] = (df['Precipitation (mm day-1)'] * df['Relative Humidity at 2 Meters (%)']) / (df['Temperature at 2 Meters (C)'] + 1)
df, kept = validate_feature(df, 'growth_conditions', 'Precipitation (mm day-1)')

print("   ✅ Índices climáticos validadas")

# 3. TRANSFORMAÇÕES
print("\n📈 3. Criando transformações...")

# Log Precipitation
df['log_precipitation'] = np.log(df['Precipitation (mm day-1)'] + 1)
df, kept = validate_feature(df, 'log_precipitation', 'Precipitation (mm day-1)')

# Temperature Squared
df['temperature_squared'] = df['Temperature at 2 Meters (C)'] ** 2
df, kept = validate_feature(df, 'temperature_squared', 'Temperature at 2 Meters (C)')

# Humidity Squared
df['humidity_squared'] = df['Relative Humidity at 2 Meters (%)'] ** 2
df, kept = validate_feature(df, 'humidity_squared', 'Relative Humidity at 2 Meters (%)')

print("   ✅ Transformações validadas")

# 4. RESUMO FINAL CORRETO
print("\n📋 RESUMO DO FEATURE ENGINEERING:")
print(f"   → Features preditoras originais: {len(original_features)}")
print(f"   → Features criadas: {len(df.columns) - len(df_original.columns)}")
print(f"   → Features removidas: {len(features_removed)}")
print(f"   → Features preditoras finais: {len(df.columns) - 1}")  # -1 para excluir Yield
print(f"   → Total de colunas: {len(df.columns)}")

if features_removed:
    print(f"\n🗑️ Features removidas por redundância:")
    for feature in features_removed:
        print(f"   → {feature}")

# Listar features preditoras finais
final_features = [col for col in df.columns if col != 'Yield']
print(f"\n✅ Features preditoras finais ({len(final_features)}):")
for i, feature in enumerate(final_features, 1):
    print(f"   {i}. {feature}")

print(f"\n🎯 Target: {target}")
print("\n✅ Feature Engineering com validação concluído!")

🔧 Iniciando Feature Engineering com validação...
�� Estado inicial:
   → Features preditoras originais: 5
   → Target: Yield
   → Total de colunas: 6

📊 1. Criando interações ambientais...
   → Correlação temp_humidity_interaction ↔ Temperature at 2 Meters (C): 0.474
   ✅ Feature mantida (correlação ≤ 0.95)
   → Correlação precip_temp_ratio ↔ Precipitation (mm day-1): 0.997
   ❌ Feature redundante (correlação > 0.95), removendo...
   → Correlação humidity_combined ↔ Specific Humidity at 2 Meters (g/kg): 0.895
   ✅ Feature mantida (correlação ≤ 0.95)
   ✅ Interações ambientais validadas

🌡️ 2. Criando índices climáticos...
   → Correlação thermal_comfort ↔ Temperature at 2 Meters (C): -0.080
   ✅ Feature mantida (correlação ≤ 0.95)
   → Correlação effective_humidity ↔ Specific Humidity at 2 Meters (g/kg): 0.895
   ✅ Feature mantida (correlação ≤ 0.95)
   → Correlação growth_conditions ↔ Precipitation (mm day-1): 0.994
   ❌ Feature redundante (correlação > 0.95), removendo...
   ✅ Índice

## 2.4 📝 Encoding de Variáveis

### Contexto e Justificativa

**Descoberta da EDA que Orienta o Encoding:**
A análise exploratória identificou que a variável 'Crop' é o **principal determinante do Yield**:
- **Oil palm fruit**: ~175k ton/ha (muito superior)
- **Rice, paddy**: ~32k ton/ha (intermediário)
- **Cocoa, beans e Rubber**: ~8k ton/ha (baixo)

**Problema Específico:**
Algoritmos de ML não processam variáveis categóricas diretamente. Necessário transformar 'Crop' em formato numérico para modelagem preditiva.

**Impacto na Modelagem:**
A variável 'Crop' é crucial para prever Yield, portanto o encoding correto é fundamental para a precisão do sistema FarmTech Solutions.

### Metodologia e Estratégia

**Abordagem Metodológica:**
Seguimos uma estratégia baseada nas características da variável categórica:

**1. One-Hot Encoding:**
- **Transformação**: Criar 4 colunas binárias (Crop_Cocoa, Crop_OilPalm, Crop_Rice, Crop_Rubber)
- **Racional**: Variável 'Crop' não tem hierarquia natural entre categorias
- **Benefício**: Evita hierarquia artificial (1, 2, 3, 4) que algoritmos podem interpretar incorretamente

**2. Alternativa Considerada - Label Encoding:**
- **Transformação**: Cocoa=0, OilPalm=1, Rice=2, Rubber=3
- **Problema**: Cria hierarquia artificial (Rubber > Rice > OilPalm > Cocoa)
- **Decisão**: Rejeitado para evitar viés hierárquico

**Justificativa das Decisões Técnicas:**
- **One-Hot Encoding**: Preserva igualdade entre culturas (sem hierarquia)
- **4 colunas binárias**: Uma para cada cultura (distribuição equilibrada: 25% cada)
- **Verificação**: Confirmar que transformação preserva informação original
- **Validação**: Garantir que não há vazamento de dados entre treino/teste

**Objetivos Específicos:**
- Transformar variável categórica 'Crop' em formato numérico para algoritmos de ML
- Preservar igualdade entre culturas sem criar hierarquia artificial
- Preparar dataset com features categóricas adequadas para modelagem preditiva

In [62]:
# =============================================================================
# ENCODING DE VARIÁVEIS CATEGÓRICAS
# =============================================================================

print("�� Iniciando Encoding de variáveis...")

# Verificar variáveis categóricas
categorical_vars = df.select_dtypes(include=['object']).columns
print(f"📋 Variáveis categóricas encontradas: {list(categorical_vars)}")

# One-Hot Encoding para variável 'Crop'
print("\n🌾 Aplicando One-Hot Encoding para 'Crop'...")

# Criar dummies
crop_dummies = pd.get_dummies(df['Crop'], prefix='Crop')

# Mostrar as novas colunas criadas
print(f"🆕 Colunas criadas: {list(crop_dummies.columns)}")

# Verificar distribuição
print("\n📊 Distribuição das categorias:")
for col in crop_dummies.columns:
    count = crop_dummies[col].sum()
    percentage = (count / len(df)) * 100
    print(f"   → {col}: {count} registros ({percentage:.1f}%)")

# Adicionar ao dataset
df = pd.concat([df, crop_dummies], axis=1)

# Remover coluna original 'Crop'
df = df.drop('Crop', axis=1)

print(f"\n✅ Encoding concluído!")
print(f"�� Dataset final: {df.shape[0]} registros, {total_features} features preditoras")
print(f"🎯 Target: {target}")
print(f"📊 Total de colunas: {df.shape[1]}")

�� Iniciando Encoding de variáveis...
📋 Variáveis categóricas encontradas: ['Crop']

🌾 Aplicando One-Hot Encoding para 'Crop'...
🆕 Colunas criadas: ['Crop_Cocoa, beans', 'Crop_Oil palm fruit', 'Crop_Rice, paddy', 'Crop_Rubber, natural']

📊 Distribuição das categorias:
   → Crop_Cocoa, beans: 39 registros (25.0%)
   → Crop_Oil palm fruit: 39 registros (25.0%)
   → Crop_Rice, paddy: 39 registros (25.0%)
   → Crop_Rubber, natural: 39 registros (25.0%)

✅ Encoding concluído!
�� Dataset final: 156 registros, 12 features preditoras
🎯 Target: Yield
📊 Total de colunas: 13


## 2.5 🔬 Normalização de Variáveis

### Contexto e Justificativa

**Descobertas da EDA que Orientam a Normalização:**
A análise exploratória revelou **alta variabilidade** entre diferentes tipos de variáveis:
- **Precipitation**: 1934-3085 mm/dia (range ~1151)
- **Temperature**: 25.56-26.81°C (range ~1.25)
- **Specific Humidity**: 17.54-18.70 g/kg (range ~1.16)
- **Relative Humidity**: 82.11-86.10% (range ~4.0)

**Problema Específico:**
Algoritmos sensíveis à escala (SVM, Neural Networks, K-means) podem dar **peso desproporcional** para variáveis com ranges maiores, comprometendo a precisão do modelo.

**Impacto na Modelagem:**
Sem normalização, Precipitation (range 1151) dominaria o modelo comparado a Temperature (range 1.25), mesmo que Temperature seja igualmente importante para prever Yield.

### Metodologia e Estratégia

**Abordagem Metodológica:**
Seguimos uma estratégia baseada nas características das variáveis numéricas:

**1. Seleção de Variáveis:**
- **Variáveis para normalizar**: Precipitation, Temperature, Specific Humidity, Relative Humidity + features criadas
- **Variáveis excluídas**: Dummies do One-Hot Encoding (já binárias: 0/1)
- **Variável alvo**: Yield mantido original (não normalizar variável alvo)

**2. StandardScaler:**
- **Transformação**: (valor - média) / desvio_padrão
- **Resultado**: Média ≈ 0, Desvio ≈ 1
- **Racional**: Método mais comum e eficaz para normalização

**3. Verificação e Validação:**
- **Confirmar normalização**: Todas as features com média ≈ 0, desvio ≈ 1
- **Preservar estrutura**: Manter informações originais dos dados
- **Salvar scaler**: Para aplicar mesma transformação em dados futuros

**Justificativa das Decisões Técnicas:**
- **StandardScaler**: Normaliza todas as features para mesma escala (média=0, desvio=1)
- **Excluir dummies**: Já estão em escala adequada (0/1)
- **Manter Yield original**: Variável alvo não deve ser normalizada
- **Salvar scaler**: Necessário para aplicar transformação em dados de produção

**Objetivos Específicos:**
- Normalizar features numéricas para algoritmos sensíveis à escala
- Garantir que todas as variáveis tenham peso igual no modelo
- Preparar dataset com features normalizadas para modelagem preditiva
- Preservar scaler para aplicação em dados futuros

In [63]:
# =============================================================================
# NORMALIZAÇÃO DE VARIÁVEIS NUMÉRICAS
# =============================================================================

print("🔬 Iniciando Normalização...")

# Selecionar variáveis numéricas (excluindo dummies E YIELD)
numeric_features = df.select_dtypes(include=[np.number]).columns
dummy_features = [col for col in df.columns if col.startswith('Crop_')]
features_to_scale = [col for col in numeric_features if col not in dummy_features and col != 'Yield']

print(f"📊 Variáveis para normalizar: {len(features_to_scale)}")
print(f"📋 Features: {list(features_to_scale)}")

# Backup dos dados originais
df_scaled = df.copy()

# Aplicar StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Normalizar features selecionadas
df_scaled[features_to_scale] = scaler.fit_transform(df[features_to_scale])

# Verificar normalização
print("\n📈 VERIFICAÇÃO DA NORMALIZAÇÃO:")
for feature in features_to_scale:
    mean_val = df_scaled[feature].mean()
    std_val = df_scaled[feature].std()
    print(f"   → {feature}:")
    print(f"     Média: {mean_val:.6f} (deveria ser ~0)")
    print(f"     Desvio: {std_val:.6f} (deveria ser ~1)")

# Verificar que Yield não foi normalizado
print(f"\n✅ Yield mantido original: {df_scaled['Yield'].mean():.2f}")

# Salvar scaler para uso futuro
import joblib
joblib.dump(scaler, '../models/scaler.pkl')

total_features = len(df_scaled.columns) - 1  # Excluir Yield
target = 'Yield'

print(f"\n✅ Normalização concluída!")
print(f"📊 Dataset normalizado: {df_scaled.shape[0]} registros, {total_features} features preditoras")
print(f"🎯 Target: {target} (escala original preservada)")
print(f"📊 Total de colunas: {df_scaled.shape[1]}")
print(f"💾 Scaler salvo em: ../models/scaler.pkl")

🔬 Iniciando Normalização...
📊 Variáveis para normalizar: 8
📋 Features: ['Precipitation (mm day-1)', 'Specific Humidity at 2 Meters (g/kg)', 'Relative Humidity at 2 Meters (%)', 'Temperature at 2 Meters (C)', 'temp_humidity_interaction', 'humidity_combined', 'thermal_comfort', 'effective_humidity']

📈 VERIFICAÇÃO DA NORMALIZAÇÃO:
   → Precipitation (mm day-1):
     Média: 0.000000 (deveria ser ~0)
     Desvio: 1.003221 (deveria ser ~1)
   → Specific Humidity at 2 Meters (g/kg):
     Média: -0.000000 (deveria ser ~0)
     Desvio: 1.003221 (deveria ser ~1)
   → Relative Humidity at 2 Meters (%):
     Média: 0.000000 (deveria ser ~0)
     Desvio: 1.003221 (deveria ser ~1)
   → Temperature at 2 Meters (C):
     Média: 0.000000 (deveria ser ~0)
     Desvio: 1.003221 (deveria ser ~1)
   → temp_humidity_interaction:
     Média: -0.000000 (deveria ser ~0)
     Desvio: 1.003221 (deveria ser ~1)
   → humidity_combined:
     Média: 0.000000 (deveria ser ~0)
     Desvio: 1.003221 (deveria ser ~1)
 

## 2.6 ✅ Dados Finais

### Contexto e Justificativa

**Transformações Aplicadas:**
Com base nas descobertas da EDA, aplicamos as seguintes transformações:
- **Análise de Outliers**: 0 outliers reais detectados (mantidos todos os dados)
- **Feature Engineering**: 4 novas features criadas (interações, índices climáticos)
- **One-Hot Encoding**: Variável 'Crop' transformada em 4 colunas binárias
- **Normalização**: 8 features padronizadas (média=0, desvio=1)

**Problema Específico:**
Necessário verificar se todas as transformações foram aplicadas corretamente e se o dataset final está pronto para as próximas etapas (clustering e modelagem).

**Impacto na Modelagem:**
Dataset mal preparado pode comprometer a precisão dos algoritmos de ML e invalidar as análises subsequentes do projeto FarmTech Solutions.

### Metodologia e Estratégia

**Abordagem Metodológica:**
Seguimos uma estratégia sistemática de validação:

**1. Verificação de Estrutura:**
- **Dimensões**: Confirmar 156 registros × 13 colunas (12 features preditoras + Yield)
- **Tipos de dados**: Verificar se todas as features estão em formato adequado
- **Integridade**: Confirmar que não há valores faltantes ou duplicatas

**2. Verificação de Qualidade:**
- **Normalização**: 8 features numéricas com média ≈ 0, desvio ≈ 1
- **Encoding**: 4 dummies do One-Hot Encoding com valores 0/1
- **Yield original**: Variável alvo mantida na escala original

**3. Verificação de Consistência:**
- **Ranges**: Confirmar que transformações preservaram informações originais
- **Distribuições**: Verificar se features derivadas fazem sentido
- **Salvamento**: Dataset final pronto para próximas etapas

**Justificativa das Decisões Técnicas:**
- **Verificação sistemática**: Garantir que todas as transformações foram aplicadas corretamente
- **Validação de qualidade**: Confirmar que dataset está adequado para algoritmos de ML
- **Documentação**: Resumir transformações para transparência metodológica
- **Salvamento**: Preparar dados para etapas subsequentes (clustering, modelagem)

**Objetivos Específicos:**
- Verificar se todas as transformações foram aplicadas corretamente
- Confirmar que dataset final está pronto para algoritmos de ML
- Documentar resumo das transformações aplicadas
- Salvar dataset preparado para próximas etapas do projeto

In [68]:
# =============================================================================
# DADOS FINAIS - VERIFICAÇÃO E SALVAMENTO
# =============================================================================

print("✅ Verificando dados finais...")

# Contar corretamente
total_features = len(df_scaled.columns) - 1  # Excluir Yield
target = 'Yield'

# Resumo final do dataset
print(f"📊 ESTRUTURA FINAL:")
print(f"   → Registros: {df_scaled.shape[0]}")
print(f"   → Features preditoras: {total_features}")
print(f"   → Features normalizadas: {len(features_to_scale)}")
print(f"   → Features categóricas (dummies): {len(dummy_features)}")
print(f"   → Variável alvo: {target} (original)")
print(f"   → Total de colunas: {df_scaled.shape[1]}")

# Verificar tipos de dados
print(f"\n�� TIPOS DE DADOS:")
print(df_scaled.dtypes.value_counts())

# Resumo estatístico das features normalizadas
print(f"\n📈 ESTATÍSTICAS DAS FEATURES NORMALIZADAS:")
print(df_scaled[features_to_scale].describe())

# Verificar Yield (não normalizado)
print(f"\n�� YIELD (VARIÁVEL ALVO):")
print(f"   → Média: {df_scaled['Yield'].mean():.2f}")
print(f"   → Desvio: {df_scaled['Yield'].std():.2f}")
print(f"   → Min: {df_scaled['Yield'].min():.2f}")
print(f"   → Max: {df_scaled['Yield'].max():.2f}")

# Salvar dataset final
df_scaled.to_csv('../data/processed/dataset_ready.csv', index=False)

print(f"\n💾 Dataset salvo em: ../data/processed/dataset_ready.csv")
print("✅ Preparação de dados concluída!")

✅ Verificando dados finais...
📊 ESTRUTURA FINAL:
   → Registros: 156
   → Features preditoras: 12
   → Features normalizadas: 8
   → Features categóricas (dummies): 4
   → Variável alvo: Yield (original)
   → Total de colunas: 13

�� TIPOS DE DADOS:
float64    8
bool       4
int64      1
Name: count, dtype: int64

📈 ESTATÍSTICAS DAS FEATURES NORMALIZADAS:
       Precipitation (mm day-1)  Specific Humidity at 2 Meters (g/kg)  \
count              1.560000e+02                          1.560000e+02   
mean               1.231921e-15                         -5.277829e-15   
std                1.003221e+00                          1.003221e+00   
min               -1.912735e+00                         -2.263223e+00   
25%               -6.360164e-01                         -5.907486e-01   
50%               -2.147065e-01                          2.284228e-01   
75%                8.026274e-01                          6.721406e-01   
max                2.077059e+00                          1

In [69]:
# =============================================================================
# RESULTADOS DA PREPARAÇÃO DE DADOS
# =============================================================================

print("\n" + "="*60)
print("🔧 RESULTADOS DA PREPARAÇÃO DE DADOS")
print("="*60)

# Calcular números dinamicamente
total_registros = len(df)
total_features = len(df.columns) - 1  # Excluir Yield
target = 'Yield'

# Contar features por tipo
original_features = ['Precipitation (mm day-1)', 'Specific Humidity at 2 Meters (g/kg)', 
                    'Relative Humidity at 2 Meters (%)', 'Temperature at 2 Meters (C)', 'Crop']
created_features = [col for col in df.columns if col not in original_features and col != target]
dummy_features = [col for col in df.columns if col.startswith('Crop_')]
normalized_features = [col for col in df.columns if col != target and not col.startswith('Crop_')]

print("📊 TRANSFORMAÇÕES APLICADAS:")
print(f"   ✅ Análise de outliers: 0 outliers reais detectados")
print(f"   ✅ Feature Engineering: {len(created_features)} novas features criadas")
print(f"   ✅ One-Hot Encoding: Variável 'Crop' transformada em {len(dummy_features)} dummies")
print(f"   ✅ Normalização: {len(normalized_features)} features padronizadas (média=0, desvio=1)")

print("\n📈 FEATURES CRIADAS:")
if created_features:
    print(f"   → Features derivadas: {', '.join(created_features)}")

print("\n📋 ESTRUTURA FINAL:")
print(f"   → Registros: {total_registros} (todos preservados)")
print(f"   → Features preditoras: {total_features}")
print(f"   → Features normalizadas: {len(normalized_features)} (prontas para ML)")
print(f"   → Variável alvo: {target} (escala original preservada)")

print("✅ Resultados da preparação de dados concluídos!")


🔧 RESULTADOS DA PREPARAÇÃO DE DADOS
📊 TRANSFORMAÇÕES APLICADAS:
   ✅ Análise de outliers: 0 outliers reais detectados
   ✅ Feature Engineering: 8 novas features criadas
   ✅ One-Hot Encoding: Variável 'Crop' transformada em 4 dummies
   ✅ Normalização: 8 features padronizadas (média=0, desvio=1)

📈 FEATURES CRIADAS:
   → Features derivadas: temp_humidity_interaction, humidity_combined, thermal_comfort, effective_humidity, Crop_Cocoa, beans, Crop_Oil palm fruit, Crop_Rice, paddy, Crop_Rubber, natural

📋 ESTRUTURA FINAL:
   → Registros: 156 (todos preservados)
   → Features preditoras: 12
   → Features normalizadas: 8 (prontas para ML)
   → Variável alvo: Yield (escala original preservada)
✅ Resultados da preparação de dados concluídos!


## 📋 Conclusões da Preparação de Dados

### Descobertas Quantitativas Críticas

**Análise de Outliers:**
- **Z-Score**: 0 outliers reais detectados (dados de qualidade)
- **IQR**: Falsos positivos em Temperature (12) e Yield (35) devido à natureza bimodal
- **Decisão estratégica**: Manter todos os dados como informação valiosa para agricultura

**Feature Engineering:**
- **4 features criadas** baseadas em correlações da EDA e conhecimento agrícola
- **Interações ambientais**: temp_humidity_interaction, humidity_combined
- **Índices climáticos**: thermal_comfort, effective_humidity
- **Validação sistemática**: 5 features removidas por redundância (correlação > 0.95)

**Encoding e Normalização:**
- **One-Hot Encoding**: 4 colunas binárias para 'Crop' (sem hierarquia artificial)
- **StandardScaler**: 8 features normalizadas (média ≈ 0, desvio ≈ 1)
- **Yield preservado**: Variável alvo mantida na escala original
- **Scaler salvo**: Para aplicação consistente em dados futuros

### Insights Agrícolas Fundamentais

**Qualidade dos Dados:**
- **Dados completos**: 0 valores faltantes, 0 duplicatas
- **Distribuição equilibrada**: 25% de cada cultura (39 registros cada)
- **Normalização perfeita**: Features numéricas em escala adequada para ML

**Preparação para Modelagem:**
- **Features derivadas**: Capturam interações entre variáveis correlacionadas
- **Índices agrícolas**: Métricas com significado para agricultores
- **Validação rigorosa**: Apenas features úteis mantidas (sem multicolinearidade)
- **Dataset final**: 156 registros × 13 colunas (12 features preditoras + Yield)

### Hipóteses Formuladas para Próximas Etapas

**H1**: Features derivadas (interações, índices) melhorarão a performance dos modelos comparado às features originais.

**H2**: Análise de clustering revelará grupos distintos baseados no tipo de cultura, não em condições ambientais.

**H3**: Algoritmos não-lineares (Random Forest, XGBoost) serão mais eficazes que modelos lineares devido às relações complexas.

**H4**: Variável 'Crop' será a feature mais importante para prever Yield, seguida pelas interações ambientais.

### Justificativa dos Próximos Passos

**Clustering:**
- **Validação da H2**: Confirmar se clusters são baseados em cultura ou condições ambientais
- **Detecção de padrões**: Identificar grupos naturais de produtividade
- **Análise de outliers**: Validar se outliers representam condições climáticas excepcionais

**Modelagem Preditiva:**
- **5 algoritmos distintos**: Testar diferentes abordagens (Linear, Tree-based, Ensemble, Neural)
- **Validação da H1 e H3**: Comparar performance com/sem features derivadas
- **Feature importance**: Validar H4 sobre importância da variável 'Crop'

**Avaliação e Interpretação:**
- **Métricas específicas**: R², RMSE, MAE para avaliação de regressão
- **Análise de resíduos**: Verificar suposições dos modelos
- **Interpretabilidade**: Explicar decisões para stakeholders agrícolas